In [19]:
# Source: https://gist.github.com/cristianpb/c41d2ac99b8a54818e8b821b1febeca7
# https://cristianpb.github.io/blog/velib-folium

import os
import folium
import pandas as pd
import numpy as np
from folium import plugins
import branca.colormap as cm
from tqdm import tqdm
import matplotlib.pyplot as plt
import json
from colour import Color
import io
from PIL import Image, ImageDraw, ImageFont
import time

import conf as cf
from datasets import getStationsInfo


In [27]:
with open(cf.DIRPATH + 'model/velib_model_list.json') as f:
    MODEL_INFO = json.load(f)
STATION_INFO = getStationsInfo()

In [28]:
station_list = [model["station_id"] for model in MODEL_INFO]

In [29]:
df_info = STATION_INFO[STATION_INFO["station_id"].isin(station_list)]
df_info

,station_id,stationCode,name,lon,lat,capacity
5,251039991,14111,Cassini - Denfert-Rochereau,2.336035,48.837526,25
7,2515829865,32017,Basilique,2.358867,48.936269,22
10,100769544,5001,Harpe - Saint-Germain,2.343670,48.851519,45
11,37874517,6003,Saint-Sulpice,2.330808,48.851654,21
15,209063434,33006,André Karman - République,2.385136,48.910399,31
...,...,...,...,...,...,...
1448,1062807847,13123,BNF - Bibliothèque Nationale de France,2.376016,48.835027,42
1450,82402482,10026,Gare de l'Est - Chateau Landon,2.362424,48.879305,59
1453,43247738,18026,Ruisseau - Ordener,2.340145,48.892995,35
1455,368766689,42004,Westermeyer - Paul Vaillant-Couturier,2.396664,48.819116,25


In [44]:
# Construction d'un dataframe à partir du json MODEL_INFO
"""
d = { "station_id": [], "y_pred": [] }
    for model in MODEL_INFO:
        d["station_id"].append(model["station_id"])
        if model["data"]["y_pred"][i*30] > 100:
            d["y_pred"].append(100.0)
        elif model["data"]["y_pred"][i*30] < 0:
            d["y_pred"].append(0.0)
        else:
            d["y_pred"].append(model["data"]["y_pred"][i*30])
    df_data = pd.DataFrame(data = d)
    
    
d = { "station_id": [], "y_test": [] }
    for model in MODEL_INFO:
        d["station_id"].append(model["station_id"])
        if model["data"]["y_test"][i*30] > 100:
            d["y_test"].append(100.0)
        elif model["data"]["y_test"][i*30] < 0:
            d["y_test"].append(0.0)
        else:
            d["y_test"].append(model["data"]["y_test"][i*30])
    df_data = pd.DataFrame(data = d)
"""

d = { "station_id": [], "error": [] }
for model in MODEL_INFO:
    d["station_id"].append(model["station_id"])
    diff = abs(model["data"]["y_pred"][0*30]) - model["data"]["y_test"][0*30]
    if diff > 100:
        d["error"].append(100.0)
    elif diff < 0:
        d["error"].append(0.0)
    else:
        d["error"].append(diff)
df_data = pd.DataFrame(data = d)
df_data

,station_id,error
0,43247738,0.000000
1,66491386,10.916632
2,100855693,27.175794
3,129050965,23.890049
4,653159330,0.000000
...,...,...
695,15462861,24.357226
696,452341449,5.733429
697,230873471,0.000000
698,39149651,0.000000


In [30]:
velibs = df_info.merge(df_data, on = "station_id")

In [31]:
velibs.error.max()

AttributeError: 'DataFrame' object has no attribute 'error'

In [32]:
# Pour faire un dégradé de couleurs
# Dans cet exemple: 10 étapes pour passer du rouge au vert

red = Color("red")
colors = list(red.range_to(Color("green").hex,10))

def red(brightness):
    #print(brightness)
    brightness = int(round(9 * brightness)) # convert from 0.0-1.0 to 0-255
    return colors[brightness]

print(colors)

[<Color red>, <Color #f13600>, <Color #e36500>, <Color #d58e00>, <Color #c7b000>, <Color #a4b800>, <Color #72aa00>, <Color #459c00>, <Color #208e00>, <Color green>]


In [33]:
# ATTENTION: Exécution assez longue (plusieurs minutes)
# Créer un GIF animé pour chaque heure de donnée
# Dans cet exemple, on utilise les données réelles (= y_test) du mardi 21 mars
frames = []

for i in range(24):
    d = { "station_id": [], "y_pred": [] }
    for model in MODEL_INFO:
        d["station_id"].append(model["station_id"])
        if model["data"]["y_pred"][i*30] > 100:
            d["y_pred"].append(100.0)
        elif model["data"]["y_pred"][i*30] < 0:
            d["y_pred"].append(0.0)
        else:
            d["y_pred"].append(model["data"]["y_pred"][i*30])
    df_data = pd.DataFrame(data = d)
    
    velibs = df_info.merge(df_data, on = "station_id")

    m = folium.Map(
        location=[48.856614, 2.3522219], 
        zoom_start=12, 
        tiles="Stamen Toner"
    )

    for k,v in velibs.iterrows():
        folium.CircleMarker(
            location = [v.lat, v.lon],
            color = "#000000",
            weight = 2,
            fill_color = red(abs(v.y_pred/100)).hex,
            fill_opacity = 1.0,
            popup = str(v.y_pred) + " % ",
            radius = 5
        ).add_to(m)
    
    img_data = m._to_png(5)
    img = Image.open(io.BytesIO(img_data))
    d1 = ImageDraw.Draw(img)
    d1.text(
        (30, 720), 
        "21/03/2023 " + str(i).zfill(2) + ":00:00", 
        fill=(64, 0, 0), 
        font = ImageFont.truetype("arial.ttf", 30)
    )
    d2 = ImageDraw.Draw(img)
    d2.text(
        (800, 20), 
        "Taux d'occupation des bornes vélibs", 
        fill=(0, 0, 0), 
        font = ImageFont.truetype("arial.ttf", 32)
    )
    frames.append(img)


frames[0].save(
    'img/prediction_2023_03_21.gif', 
    format = 'GIF',
    append_images = frames[1:],
    save_all = True,
    duration = 500,
    loop = 0
)

In [28]:
# Code d'origine pour sauvegarder les images intermédiaires en png, puis toutes les récupérer après pour faire un gif

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('image.png')

# Create the frames
frames = []
imgs = glob.glob("*.png")
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)
 
# Save into a GIF file that loops forever
frames[0].save('png_to_gif.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=300, loop=0)

In [10]:
# Pour faire un dégradé de couleurs
# Dans cet exemple: 2 étapes pour passer du noir au blanc

white = Color("white")
colors = list(white.range_to(Color("black").hex,2))

def black_white(brightness):
    #print(brightness)
    brightness = int(round(brightness)) # convert from 0.0-1.0 to 0-255
    return colors[brightness]

print(colors)

[<Color white>, <Color black>]


In [13]:
d = { "station_id": [], "error": [] }
for model in MODEL_INFO:
    d["station_id"].append(model["station_id"])
    diff = abs(model["data"]["y_pred"][0*30]) - model["data"]["y_test"][0*30]
    if diff > 10:
        d["error"].append(1.0)
    else:
        d["error"].append(0.0)
df_data = pd.DataFrame(data = d)
df_data
df_info

,station_id,stationCode,name,lon,lat,capacity
5,251039991,14111,Cassini - Denfert-Rochereau,2.336035,48.837526,25
7,2515829865,32017,Basilique,2.358867,48.936269,22
10,100769544,5001,Harpe - Saint-Germain,2.343670,48.851519,45
11,37874517,6003,Saint-Sulpice,2.330808,48.851654,21
15,209063434,33006,André Karman - République,2.385136,48.910399,31
...,...,...,...,...,...,...
1448,1062807847,13123,BNF - Bibliothèque Nationale de France,2.376016,48.835027,42
1450,82402482,10026,Gare de l'Est - Chateau Landon,2.362424,48.879305,59
1453,43247738,18026,Ruisseau - Ordener,2.340145,48.892995,35
1455,368766689,42004,Westermeyer - Paul Vaillant-Couturier,2.396664,48.819116,25


In [21]:
start_tm = time.time()

frames = []

for i in range(24):
    start_tm_i = time.time()
    
    d = { "station_id": [], "error": [] }
    for model in MODEL_INFO:
        d["station_id"].append(model["station_id"])
        diff = abs(model["data"]["y_pred"][i*30]) - model["data"]["y_test"][i*30]
        if diff > 10:
            d["error"].append(1.0)
        else:
            d["error"].append(0.0)
    df_data = pd.DataFrame(data = d)

    velibs = df_info.merge(df_data, on = "station_id")
    
    print(str(i).zfill(2) + ":", "Dataset velibs terminé")

    m = folium.Map(
        location=[48.856614, 2.3522219], 
        zoom_start=12, 
        #tiles="Stamen Toner",
    )
    
    print(str(i).zfill(2) + ":", "Map Folium créée")

    for k,v in velibs.iterrows():
        folium.CircleMarker(
            location = [v.lat, v.lon],
            color = "#000000",
            weight = 1,
            fill_color = black_white(v.error).hex,
            fill_opacity = 1.0,
            popup = str(v.stationCode) + "\n" + str(v.error),
            radius = 5
        ).add_to(m)
    
    print(str(i).zfill(2) + ":", "Cercles ajoutés")

    img_data = m._to_png(5)
    img = Image.open(io.BytesIO(img_data))
    
    print(str(i).zfill(2) + ":", "Image mise en png et sauvegardée")
    
    d1 = ImageDraw.Draw(img)
    d1.text(
        (30, 720), 
        "21/03/2023 " + str(i).zfill(2) + ":00:00", 
        fill=(64, 0, 0), 
        font = ImageFont.truetype("arial.ttf", 30)
    )    
    d2 = ImageDraw.Draw(img)
    d2.text(
        (800, 20), 
        "Blanc = Error < 10 point, Noir sinon", 
        fill=(0, 0, 0), 
        font = ImageFont.truetype("arial.ttf", 32)
    )
    
    print(str(i).zfill(2) + ":", "Textes ajoutés")    
    frames.append(img)
    
    print(str(i).zfill(2) + ":" + str(time.time() - start_tm_i) + "s")
    
print("Fin de la boucle")


frames[0].save(
    'errors_2023_03_21.gif', 
    format = 'GIF',
    append_images = frames[1:],
    save_all = True,
    duration = 500,
    loop = 0
)
    
print("GIF animé créé en "+ str(time.time() - start_tm) + "s")

00: Dataset velibs terminé
00: Map Folium créée
00: Cercles ajoutés
00: Image mise en png et sauvegardée
00: Textes ajoutés
00:19.69987416267395s
01: Dataset velibs terminé
01: Map Folium créée
01: Cercles ajoutés
01: Image mise en png et sauvegardée
01: Textes ajoutés
01:19.868451833724976s
02: Dataset velibs terminé
02: Map Folium créée
02: Cercles ajoutés
02: Image mise en png et sauvegardée
02: Textes ajoutés
02:20.358139038085938s
03: Dataset velibs terminé
03: Map Folium créée
03: Cercles ajoutés
03: Image mise en png et sauvegardée
03: Textes ajoutés
03:21.615909337997437s
04: Dataset velibs terminé
04: Map Folium créée
04: Cercles ajoutés
04: Image mise en png et sauvegardée
04: Textes ajoutés
04:21.629996061325073s
05: Dataset velibs terminé
05: Map Folium créée
05: Cercles ajoutés
05: Image mise en png et sauvegardée
05: Textes ajoutés
05:20.90292525291443s
06: Dataset velibs terminé
06: Map Folium créée
06: Cercles ajoutés
06: Image mise en png et sauvegardée
06: Textes ajou

In [23]:
STATION_INFO["capacity"].max()

74

In [25]:
m = folium.Map(
    location=[48.856614, 2.3522219], 
    zoom_start=12, 
    #tiles="Stamen Toner"
)

for k,v in STATION_INFO.iterrows():
    folium.CircleMarker(
        location = [v.lat, v.lon],
        color = "#000000",
        weight = 2,
        fill_color = "#000000",
        fill_opacity = 0.3,
        #popup = str(v.y_test) + " % ",
        radius = 1 + 5 * (v.capacity / 74)
    ).add_to(m)

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('img/stations_list.png')